In [1]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import os

In [2]:
averages_multi_queue_current_state = pd.read_csv('results_directory/same_patient_profiles/multi_queue/current_state/averages_data.csv')
averages_single_queue_current_state = pd.read_csv('results_directory/same_patient_profiles/single_queue/current_state/averages_data.csv')
averages_multi_queue_80_percent_nurse_for_transplant = pd.read_csv('results_directory/same_patient_profiles/multi_queue_v2/current_state/averages_data.csv')
averages_single_queue_80_percent_nurse_for_transplant = pd.read_csv('results_directory/same_patient_profiles/single_queue_v2/current_state/averages_data.csv')
averages_single_queue_personalized_state = pd.read_csv('results_directory/same_patient_profiles/single_queue/personalized/averages_data.csv')
averages_multi_queue_personalized_state = pd.read_csv('results_directory/same_patient_profiles/multi_queue/personalized/averages_data.csv')
averages_single_queue_personalized_state_80_percent_nurse_for_transplant = pd.read_csv('results_directory/same_patient_profiles/single_queue_v2/personalized/averages_data.csv')
averages_multi_queue_personalized_state_80_percent_nurse_for_transplant = pd.read_csv('results_directory/same_patient_profiles/multi_queue_v2/personalized/averages_data.csv')

# I want to create two dfs like the ones loaded but without columns containing the word "nurse"
nurse_columns_multi_queue = [col for col in averages_multi_queue_current_state.columns if "nurse" in col]
nurse_columns_single_queue = [col for col in averages_single_queue_current_state.columns if "nurse" in col]
nurse_columns_multi_queue_80_percent_nurse_for_transplant = [col for col in averages_multi_queue_80_percent_nurse_for_transplant.columns if "nurse" in col]
nurse_columns_single_queue_80_percent_nurse_for_transplant = [col for col in averages_single_queue_80_percent_nurse_for_transplant.columns if "nurse" in col]
nurse_columns_multi_queue_personalized_state = [col for col in averages_multi_queue_personalized_state.columns if "nurse" in col]
nurse_columns_single_queue_personalized_state = [col for col in averages_single_queue_personalized_state.columns if "nurse" in col]
nurse_columns_multi_queue_personalized_state_80_percent_nurse_for_transplant = [col for col in averages_multi_queue_personalized_state_80_percent_nurse_for_transplant.columns if "nurse" in col]
nurse_columns_single_queue_personalized_state_80_percent_nurse_for_transplant = [col for col in averages_single_queue_personalized_state_80_percent_nurse_for_transplant.columns if "nurse" in col]

no_nurse_multi_queue = averages_multi_queue_current_state.drop(columns=nurse_columns_multi_queue)
no_nurse_single_queue = averages_single_queue_current_state.drop(columns=nurse_columns_single_queue)
no_nurse_multi_queue_80_percent_nurse_for_transplant = averages_multi_queue_80_percent_nurse_for_transplant.drop(columns=nurse_columns_multi_queue_80_percent_nurse_for_transplant)
no_nurse_single_queue_80_percent_nurse_for_transplant = averages_single_queue_80_percent_nurse_for_transplant.drop(columns=nurse_columns_single_queue_80_percent_nurse_for_transplant)
no_nurse_multi_queue_personalized_state = averages_multi_queue_personalized_state.drop(columns=nurse_columns_multi_queue_personalized_state)
no_nurse_single_queue_personalized_state = averages_single_queue_personalized_state.drop(columns=nurse_columns_single_queue_personalized_state)
no_nurse_multi_queue_personalized_state_80_percent_nurse_for_transplant = averages_multi_queue_personalized_state_80_percent_nurse_for_transplant.drop(columns=nurse_columns_multi_queue_personalized_state_80_percent_nurse_for_transplant)
no_nurse_single_queue_personalized_state_80_percent_nurse_for_transplant = averages_single_queue_personalized_state_80_percent_nurse_for_transplant.drop(columns=nurse_columns_single_queue_personalized_state_80_percent_nurse_for_transplant)

### Here we compare both simulations which were ran with identical patient profiles

### Essentially, we have a sample of pairs $$(X_1, Y_1), (X_2, Y_2),\dots, (X_{100}, Y_{100})$$
### We can now create a sample of differences $$D_i = (X_1 - Y_i)$$
### Now $D_1, D_2, \dots D_{100}$ are i.i.d. 
### Additionally, $$\mathbf{E}(D_i) = \mathbf{E}(X_i) - \mathbf{E}(Y_i)$$
### Our only assumption from here on out will be that $D_i\sim\mathcal{N}(\mathbf{E}(D_i), \mathbf{Var}(D_i))$
### This is not a groundless assumption, since, for sufficiently large $n$, $X$ and $Y$, which are averages, should be approximatelly normally distributed. as such, their difference, as a linear function of two normally distributed variables, would also be normally distributed.

#### Credit to Technion statistics 1 course - presentation 10

In [3]:
current_vs_single_current_state = averages_multi_queue_current_state - averages_single_queue_current_state
current_vs_multi_queue_80_percent_nurse_for_transplant = averages_multi_queue_current_state - averages_multi_queue_80_percent_nurse_for_transplant
current_vs_single_queue_80_percent_nurse_for_transplant = averages_single_queue_current_state - averages_single_queue_80_percent_nurse_for_transplant
current_vs_single_queue_personalized_state = averages_single_queue_current_state - averages_single_queue_personalized_state
current_vs_multi_queue_personalized_state = averages_multi_queue_current_state - averages_multi_queue_personalized_state
current_vs_single_queue_personalized_state = averages_single_queue_current_state - averages_single_queue_personalized_state
current_vs_multi_queue_personalized_state_80_percent_nurse_for_transplant = averages_multi_queue_current_state - averages_multi_queue_personalized_state_80_percent_nurse_for_transplant
current_vs_single_queue_personalized_state_80_percent_nurse_for_transplant = averages_single_queue_current_state - averages_single_queue_personalized_state_80_percent_nurse_for_transplant

### Our hypothesis $H_0$ is - $\mu_d = d_0$.
### In our case $n = 100$; $d_0 = 0$

### We make two observations:
### - Under $H_0$: $T = \frac{\bar{D}-d_0}{\frac{S_D}{\sqrt{n}}}\sim t_{(n-1)}$
### - $S_D^{2} = \frac{1}{n-1} \sum_{i=1}^{n} \left(D_i - \bar{D}\right)^2$

### Thus, a confidence interval at confidence level $1 - \alpha$ is:
### $$\left[\bar{D} \pm t_{(n-1),1-\frac{\alpha}{2}}\frac{S_D}{\sqrt{n}}\right]$$

#### Credit to Technion statistics 1 course - presentation 10

In [4]:
def S_D(df, column_name):
    """
    Calculate the sample standard deviation for a given column in a DataFrame.
    """
    return np.std(df[column_name], ddof=1) # ddof=1 to ensure the sample standard deviation is an unbiased estimator of the population standard deviation

def D_bar(df, column_name):
    """
    Calculate the sample mean for a given column in a DataFrame.
    """
    return np.mean(df[column_name])

def calculate_confidence_interval(df, column_name, confidence_level=0.95):
    """
    Calculate the confidence interval for a given column in a DataFrame.
    
    Args:
        df (pd.DataFrame): The DataFrame containing the data.
        column_name (str): The name of the column to calculate the confidence interval for.
        confidence_level (float): The confidence level for the interval (default is 0.95).
        
    Returns:
        tuple: A tuple containing the lower and upper bounds of the confidence interval.
    """

    n = len(df)
    t_value = stats.t.ppf(1 - (1 - confidence_level) / 2, n - 1)
    lower_bound = D_bar(df, column_name) - t_value * S_D(df, column_name) / np.sqrt(n)
    upper_bound = D_bar(df, column_name) + t_value * S_D(df, column_name) / np.sqrt(n)
    return lower_bound, upper_bound

def does_null_hypothesis_hold(df, column_name, confidence_level=0.95):
    """
    Check if the null hypothesis holds for a given column in a DataFrame.
    """
    lower_bound, upper_bound = calculate_confidence_interval(df, column_name, confidence_level)
    return lower_bound <= 0 <= upper_bound

def does_null_hypothesis_hold_for_all_columns(df, confidence_level=0.95):
    """
    Check if the null hypothesis holds for all columns in a DataFrame.
    """
    for column in df.columns:
        if not does_null_hypothesis_hold(df, column, confidence_level):
            return False
    return True

def csv_of_results(df, column_name, confidence_level=0.95):
    # Create results dataframe with proper columns
    results_df = pd.DataFrame(columns=['lower_bound', 'upper_bound', 'null_hypothesis_holds'])
    
    # Create the directory if it doesn't exist
    output_dir = "results_directory/statistical_analysis"
    os.makedirs(output_dir, exist_ok=True)
    
    for column in df.columns:
        try:
            lower_bound, upper_bound = calculate_confidence_interval(df, column, confidence_level)
            null_hypothesis_result = does_null_hypothesis_hold(df, column, confidence_level)
            
            # Add row using loc with proper indexing
            results_df.loc[column] = {
                'lower_bound': lower_bound,
                'upper_bound': upper_bound, 
                'null_hypothesis_holds': null_hypothesis_result
            }
        except Exception as e:
            print(f"Error processing column {column}: {e}")
            # Add row with NaN values if there's an error
            results_df.loc[column] = {
                'lower_bound': np.nan,
                'upper_bound': np.nan,
                'null_hypothesis_holds': np.nan
            }
            # Save all results
    results_df.to_csv(f"{output_dir}/{column_name}.csv")

    # Filter rows where null hypothesis is rejected (i.e., False)
    rejected_df = results_df[results_df['null_hypothesis_holds'] == False]
    
    # Display only rejected rows in Jupyter Notebook
    from IPython.display import display
    display(rejected_df)

    return rejected_df

    return results_df

In [5]:
csv_of_results(current_vs_single_current_state, "differences_current_vs_single_current_state")

,lower_bound,upper_bound,null_hypothesis_holds
general_nurse_station_busy_avg,NaN,NaN,False
general_nurse_station_queue_length_avg,NaN,NaN,False
general_nurse_station_scheduled_vs_actual_time_diff_avg,NaN,NaN,False
general_nurse_station_wait_time_avg,NaN,NaN,False
leukemia_doctor_1_queue_length_avg,-0.893947,-0.293619,False
leukemia_doctor_1_wait_time_avg,-32.525878,-9.228536,False
nurse_station_1_busy_avg,NaN,NaN,False
nurse_station_1_queue_length_avg,NaN,NaN,False
nurse_station_1_scheduled_vs_actual_time_diff_avg,NaN,NaN,False
nurse_station_1_wait_time_avg,NaN,NaN,False


,lower_bound,upper_bound,null_hypothesis_holds
general_nurse_station_busy_avg,NaN,NaN,False
general_nurse_station_queue_length_avg,NaN,NaN,False
general_nurse_station_scheduled_vs_actual_time_diff_avg,NaN,NaN,False
general_nurse_station_wait_time_avg,NaN,NaN,False
leukemia_doctor_1_queue_length_avg,-0.893947,-0.293619,False
leukemia_doctor_1_wait_time_avg,-32.525878,-9.228536,False
nurse_station_1_busy_avg,NaN,NaN,False
nurse_station_1_queue_length_avg,NaN,NaN,False
nurse_station_1_scheduled_vs_actual_time_diff_avg,NaN,NaN,False
nurse_station_1_wait_time_avg,NaN,NaN,False


In [6]:
csv_of_results(current_vs_multi_queue_80_percent_nurse_for_transplant, "differences_current_vs_multi_queue_80_percent_nurse_for_transplant")

,lower_bound,upper_bound,null_hypothesis_holds
leukemia_doctor_2_regular_patients_total_processing_time_avg,-25.852049,-2.390662,False


,lower_bound,upper_bound,null_hypothesis_holds
leukemia_doctor_2_regular_patients_total_processing_time_avg,-25.852049,-2.390662,False


In [7]:
csv_of_results(current_vs_single_queue_80_percent_nurse_for_transplant, "differences_current_vs_single_queue_80_percent_nurse_for_transplant")

,lower_bound,upper_bound,null_hypothesis_holds
leukemia_doctor_1_complex_patients_total_processing_time_avg,0.128604,27.033467,False


,lower_bound,upper_bound,null_hypothesis_holds
leukemia_doctor_1_complex_patients_total_processing_time_avg,0.128604,27.033467,False


In [8]:
csv_of_results(current_vs_single_queue_personalized_state, "differences_current_vs_single_queue_personalized_state")

,lower_bound,upper_bound,null_hypothesis_holds
secretary_station_queue_length_avg,0.003910,0.081507,False
leukemia_doctor_2_regular_patients_total_processing_time_avg,-24.765919,-0.425267,False
general_nurse_station_scheduled_vs_actual_time_diff_avg,-2.795394,-0.359443,False


,lower_bound,upper_bound,null_hypothesis_holds
secretary_station_queue_length_avg,0.003910,0.081507,False
leukemia_doctor_2_regular_patients_total_processing_time_avg,-24.765919,-0.425267,False
general_nurse_station_scheduled_vs_actual_time_diff_avg,-2.795394,-0.359443,False


In [9]:
csv_of_results(current_vs_multi_queue_personalized_state, "differences_current_vs_multi_queue_personalized_state")

,lower_bound,upper_bound,null_hypothesis_holds
leukemia_doctor_2_busy_avg,-0.069299,-0.004249,False
leukemia_doctor_1_complex_patients_total_processing_time_avg,-32.517911,-6.842612,False
leukemia_doctor_1_regular_patients_total_processing_time_avg,-42.259437,-15.197663,False
leukemia_doctor_2_complex_patients_total_processing_time_avg,-40.347032,-4.716770,False
leukemia_doctor_2_regular_patients_total_processing_time_avg,-33.365906,-9.160585,False
leukemia_doctor_2_scheduled_vs_actual_time_diff_avg,-26.806809,-4.839426,False
transplant_doctor_2_scheduled_vs_actual_time_diff_avg,-29.929523,-0.699683,False


,lower_bound,upper_bound,null_hypothesis_holds
leukemia_doctor_2_busy_avg,-0.069299,-0.004249,False
leukemia_doctor_1_complex_patients_total_processing_time_avg,-32.517911,-6.842612,False
leukemia_doctor_1_regular_patients_total_processing_time_avg,-42.259437,-15.197663,False
leukemia_doctor_2_complex_patients_total_processing_time_avg,-40.347032,-4.716770,False
leukemia_doctor_2_regular_patients_total_processing_time_avg,-33.365906,-9.160585,False
leukemia_doctor_2_scheduled_vs_actual_time_diff_avg,-26.806809,-4.839426,False
transplant_doctor_2_scheduled_vs_actual_time_diff_avg,-29.929523,-0.699683,False


In [10]:
csv_of_results(current_vs_single_queue_personalized_state, "differences_current_vs_single_queue_personalized_state")

,lower_bound,upper_bound,null_hypothesis_holds
secretary_station_queue_length_avg,0.003910,0.081507,False
leukemia_doctor_2_regular_patients_total_processing_time_avg,-24.765919,-0.425267,False
general_nurse_station_scheduled_vs_actual_time_diff_avg,-2.795394,-0.359443,False


,lower_bound,upper_bound,null_hypothesis_holds
secretary_station_queue_length_avg,0.003910,0.081507,False
leukemia_doctor_2_regular_patients_total_processing_time_avg,-24.765919,-0.425267,False
general_nurse_station_scheduled_vs_actual_time_diff_avg,-2.795394,-0.359443,False


In [11]:
csv_of_results(current_vs_multi_queue_personalized_state_80_percent_nurse_for_transplant, "differences_current_vs_multi_queue_personalized_state_80_percent_nurse_for_transplant")

,lower_bound,upper_bound,null_hypothesis_holds
nurse_station_1_queue_length_avg,0.000732,0.222272,False
nurse_station_2_busy_avg,-0.099855,-0.001205,False
leukemia_doctor_2_regular_patients_total_processing_time_avg,-27.920669,-6.330154,False
nurse_station_2_scheduled_vs_actual_time_diff_avg,-15.113718,-0.216901,False


,lower_bound,upper_bound,null_hypothesis_holds
nurse_station_1_queue_length_avg,0.000732,0.222272,False
nurse_station_2_busy_avg,-0.099855,-0.001205,False
leukemia_doctor_2_regular_patients_total_processing_time_avg,-27.920669,-6.330154,False
nurse_station_2_scheduled_vs_actual_time_diff_avg,-15.113718,-0.216901,False


In [12]:
csv_of_results(current_vs_single_queue_personalized_state_80_percent_nurse_for_transplant, "differences_current_vs_single_queue_personalized_state_80_percent_nurse_for_transplant")

,lower_bound,upper_bound,null_hypothesis_holds
other_patients_total_processing_time_avg,-7.032470,-0.926684,False
general_nurse_station_scheduled_vs_actual_time_diff_avg,-3.521903,-0.803779,False


,lower_bound,upper_bound,null_hypothesis_holds
other_patients_total_processing_time_avg,-7.032470,-0.926684,False
general_nurse_station_scheduled_vs_actual_time_diff_avg,-3.521903,-0.803779,False
